In [108]:
import pandas as pd
import json
import os

import warnings
warnings.filterwarnings('ignore')

In [76]:
params = [
    "base_solution", "heuristic",
    "search", "iterations", "temp_coef", "population_size",
    "mutation_probability","tabu_queue_coef",
    "swap_any_to_generate", "TotalDistance",
    "TotalTime", "UnusedTrucksCount", "OrphansCount", "execution_time",
    "TotalFuel"
]

def generate_data(folder):
    df_data = {key: [] for key in params}
    df_data["TrucksCount"] = []
    for output in os.listdir(folder):
        with open(f"{folder}/{output}") as f:
            json_data = json.load(f)
        df_data["TrucksCount"].append(len(json_data["Trucks"]))
        for key in params:
            if key not in json_data:
                df_data[key].append(None)
            else:
                df_data[key].append(json_data[key])
    return pd.DataFrame(data=df_data)

df = generate_data("results_lviv")

In [77]:
df.to_csv("lviv_results.csv")

In [78]:
df = pd.read_csv("lviv_results.csv")
df.head()

,Unnamed: 0,base_solution,heuristic,search,iterations,temp_coef,population_size,mutation_probability,tabu_queue_coef,swap_any_to_generate,TotalDistance,TotalTime,UnusedTrucksCount,OrphansCount,execution_time,TotalFuel,TrucksCount
0,0,SIMPLE_GREEDY,ANY_EXPORTATION_PROB1,TABU,500.0,NaN,NaN,NaN,0.5,NaN,1822.1130,4.910227,0,0,71.572512,455.528250,20
1,1,SIMPLE_GREEDY,MOVE_IN_EXPORTATION,ANNEALING,5000.0,0.00001,NaN,NaN,NaN,NaN,1741.2769,12.861653,0,0,338.529538,435.319225,10
2,2,SIMPLE_GREEDY,REMOVE_LOOPS_SWAP_ANY_EXPORTATION,GENETIC,200.0,NaN,10.0,0.1,NaN,NaN,1805.0859,4.795727,0,0,128.327872,451.271475,20
3,3,SIMPLE_GREEDY,REMOVE_LOOPS_SWAP_ANY_EXPORTATION,GENETIC,400.0,NaN,10.0,0.3,NaN,NaN,1777.5801,6.672147,0,0,219.698072,444.395025,15
4,4,SIMPLE_GREEDY,REMOVE_LOOPS_MOVE_EXPORTATION,ANNEALING,3000.0,0.00001,NaN,NaN,NaN,NaN,1803.9392,4.799982,0,0,265.007310,450.984800,20


In [84]:
df[(df["search"] == "TABU") | (df["search"] == "NO_SEARCH")]

,Unnamed: 0,base_solution,heuristic,search,iterations,temp_coef,population_size,mutation_probability,tabu_queue_coef,swap_any_to_generate,TotalDistance,TotalTime,UnusedTrucksCount,OrphansCount,execution_time,TotalFuel,TrucksCount


In [112]:
def get_plot_data(df, search, trucks_count, base_solution="SIMPLE_GREEDY"):
    return df[((df["search"] == search) | (df["search"] == "NO_SEARCH")) &
              (df["TrucksCount"] == trucks_count) &
              (df["base_solution"] == base_solution)
             ]

tabu_greedy_10 = get_plot_data(df, "TABU", 10)
tabu_greedy_15 = get_plot_data(df, "TABU", 15)
tabu_greedy_20 = get_plot_data(df, "TABU", 20)
annealing_greedy_10 = get_plot_data(df, "ANNEALING", 10)
annealing_greedy_15 = get_plot_data(df, "ANNEALING", 15)
annealing_greedy_20 = get_plot_data(df, "ANNEALING", 20)
genetic_greedy_10 = get_plot_data(df, "GENETIC", 10)
genetic_greedy_15 = get_plot_data(df, "GENETIC", 15)
genetic_greedy_20 = get_plot_data(df, "GENETIC", 20)

In [167]:
import altair as alt
import numpy as np
def plot_results(data, tooltips, title=""):
    title = alt.TitleParams(
        title,
        fontSize=20,
        anchor="middle",
    )

    minDistance = data["TotalDistance"].min()
    maxDistance = data[data["search"] == "NO_SEARCH"]["TotalDistance"].max()
    data = data[data["TotalDistance"] <= maxDistance]
    minTime = data[data["search"] == "NO_SEARCH"]["execution_time"].min()
    maxTime = data["execution_time"].max()

    data["Relation"] = (maxDistance - data["TotalDistance"]) / (data["execution_time"] - minTime + 0.00001)

    y = np.arange(minTime - 10, maxTime + 10, 0.1)
    max_relation = data["Relation"].max()
    y = y[- max_relation * (y - minTime) + maxDistance < maxDistance + 10]
    y = y[- max_relation * (y - minTime) + maxDistance > minDistance - 10]
    source = pd.DataFrame({
      'x': - max_relation * (y - minTime) + maxDistance,
      'y': y
    })

    best_relation_line = alt.Chart(source).mark_line().encode(
        x=alt.X('x', scale=alt.Scale(domain=[maxDistance + 10, minDistance - 10])),
        y=alt.Y("y")
    )

    points = alt.Chart(data, title=title).mark_circle(
        size=60
    ).encode(
        x=alt.X(
            "TotalDistance",
            scale=alt.Scale(domain=[maxDistance + 10, minDistance - 10]),
            title="Total distance"
        ),
        y=alt.Y(
            "execution_time",
            scale=alt.Scale(domain=[minTime - 10, maxTime + 10]),
            title="Execution time"
        ),
        color=alt.Color("search", title="Search"),
        tooltip=tooltips
    )

    return points + best_relation_line


In [168]:
tabu_tooltips = [
    alt.Tooltip("heuristic", title="Heuristic"),
    alt.Tooltip("search", title="Search"),
    alt.Tooltip("base_solution", title="Base solution"),
    alt.Tooltip("iterations", title="Iterations"),
    alt.Tooltip("TotalDistance", title="Total distance"),
    alt.Tooltip("execution_time", title="Execution time"),
    alt.Tooltip("tabu_queue_coef", title="Queue coef"),
]

annealing_tooltips = [
    alt.Tooltip("heuristic", title="Heuristic"),
    alt.Tooltip("search", title="Search"),
    alt.Tooltip("base_solution", title="Base solution"),
    alt.Tooltip("iterations", title="Iterations"),
    alt.Tooltip("TotalDistance", title="Total distance"),
    alt.Tooltip("execution_time", title="Execution time"),
    alt.Tooltip("temp_coef", title="Temp coef"),
]

genetic_tooltips = [
    alt.Tooltip("heuristic", title="Heuristic"),
    alt.Tooltip("search", title="Search"),
    alt.Tooltip("base_solution", title="Base solution"),
    alt.Tooltip("iterations", title="Iterations"),
    alt.Tooltip("TotalDistance", title="Total distance"),
    alt.Tooltip("execution_time", title="Execution time"),
    alt.Tooltip("population_size", title="Population size"),
    alt.Tooltip("mutation_probability", title="Mutation probability"),
]


In [169]:
plot_results(tabu_greedy_10, tabu_tooltips, "Tabu results for 10 trucks").properties(
    width=600, height=400, background="#F9F9F9"
)

alt.LayerChart(...)

In [171]:
plot_results(annealing_greedy_10, annealing_tooltips, "Simulated annealing results for 10 trucks").properties(
    width=600, height=400, background="#F9F9F9"
)

alt.LayerChart(...)

In [172]:
plot_results(genetic_greedy_10, genetic_tooltips, "Genetic algorithm results for 10 trucks").properties(
    width=600, height=400, background="#F9F9F9"
)

alt.LayerChart(...)

In [173]:
plot_results(tabu_greedy_15, tabu_tooltips, "Tabu results for 15 trucks").properties(
    width=600, height=400, background="#F9F9F9"
)

alt.LayerChart(...)

In [174]:
plot_results(annealing_greedy_15, annealing_tooltips, "Simulated annealing results for 15 trucks").properties(
    width=600, height=400, background="#F9F9F9"
)

alt.LayerChart(...)

In [175]:
plot_results(genetic_greedy_15, genetic_tooltips, "Genetic algorithm results for 15 trucks").properties(
    width=600, height=400, background="#F9F9F9"
)

alt.LayerChart(...)

In [176]:
plot_results(tabu_greedy_20, tabu_tooltips, "Tabu results for 20 trucks").properties(
    width=600, height=400, background="#F9F9F9"
)

alt.LayerChart(...)

In [177]:
plot_results(annealing_greedy_20, annealing_tooltips, "Simulated annealing results for 20 trucks").properties(
    width=600, height=400, background="#F9F9F9"
)

alt.LayerChart(...)

In [178]:
plot_results(genetic_greedy_20, annealing_tooltips, "Genetic algorithm results for 20 trucks").properties(
    width=600, height=400, background="#F9F9F9"
)

alt.LayerChart(...)